# Reproducibility Lab: Functional Connectivity and Depression

## HYPOTHESIS-DRIVEN ANALYSIS

## Learning Objectives

By the end of this lab, you will be able to:
- Formulate a hypothesis about brain-behavior relationships based on prior literature
- Use analytical tools (covariates, outliers, subgroups) to refine your analysis
- Pre-register an analysis plan before seeing your results
- Conduct a hypothesis-driven functional connectivity analysis with Bonferroni correction
- Validate findings in an independent dataset
- Present rigorous, replicable findings to the class

---

## Part 1: Setup and Load Data

In [1]:
# Install nilearn for brain visualizations and download data files
import subprocess, sys
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'nilearn', '-q'])

import os
import urllib.request

base_url = 'https://raw.githubusercontent.com/cmahlen/python-stats-demo/main/'
files_needed = [
    'lab_helpers.py',
    'atlas_labels.txt',
    'data/roi_mni_coords.npy',
    'data/depression_discovery.npz',
    'data/depression_validation.npz',
]

os.makedirs('data', exist_ok=True)
for f in files_needed:
    if not os.path.exists(f):
        print(f'Downloading {f}...')
        urllib.request.urlretrieve(base_url + f, f)

print('Setup complete!')

Setup complete!



[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: pip3.11 install --upgrade pip


In [2]:
import lab_helpers as helpers
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# Load the discovery dataset
helpers.load_dataset('depression', 'discovery')

Loaded depression discovery dataset:
  200 subjects
  216 brain regions (ROIs)
  23,220 connectivity edges
  Outcome variable: PHQ9
  Other variables: Age, Sex, BMI, HRV, Sleep_Quality, Physical_Activity, Caffeine_mg, Stress_Level, Rumination, Loneliness, Social_Support, Neuroticism, Self_Esteem, Alcohol_drinks, Screen_Time, Social_Media, Education_yrs, Income_k


This dataset is split into two halves: a **discovery set** (which you just loaded) and a **validation set** (which you will use on Day 2). You will find your results in the discovery set first, and then test whether they replicate in the validation set -- a completely independent sample.

---

## Part 2: Peek at the Data

Before testing your hypothesis, look at the actual data. This is a crucial first step in any analysis.

In [5]:
# What variables do we have? Print descriptions and units for each one.
helpers.describe_variables()

Variables in the depression dataset:
Variable                       Description
---------------------------------------------------------------------------
PHQ9                           Depression severity (PHQ-9 questionnaire, 0-27)
Age                            Age in years
Sex                            Biological sex (0 = female, 1 = male)
BMI                            Body mass index (kg/m^2)
HRV                            Heart rate variability (RMSSD, ms)
Sleep_Quality                  Sleep quality (PSQI, 0-21; higher = worse sleep)
Physical_Activity              Physical activity (minutes/week)
Caffeine_mg                    Caffeine intake (mg/day)
Stress_Level                   Perceived stress (PSS, 0-40)
Rumination                     Rumination (RRS, 22-88)
Loneliness                     Loneliness (UCLA scale, 20-80)
Social_Support                 Perceived social support (MSPSS, 12-60)
Neuroticism                    Neuroticism (NEO, 0-48)
Self_Esteem                

In [13]:
# Look at the first few rows of behavioral data
behavior = helpers.get_behavior()
behavior = behavior.astype(float).round(3)
behavior.head()

,PHQ9,Age,Sex,BMI,HRV,Sleep_Quality,Physical_Activity,Caffeine_mg,Stress_Level,Rumination,Loneliness,Social_Support,Neuroticism,Self_Esteem,Alcohol_drinks,Screen_Time,Social_Media,Education_yrs,Income_k
0,3.749,38.725,0.0,28.214,68.034,10.686,90.253,128.859,18.203,40.047,46.022,27.775,26.393,17.924,14.043,4.635,101.669,12.066,53.378
1,7.977,21.725,0.0,22.772,47.019,9.261,13.579,127.974,24.378,40.902,45.864,31.934,39.152,15.379,13.300,5.053,123.524,16.200,46.492
2,3.518,22.310,1.0,17.552,49.828,12.050,106.348,145.061,21.738,36.926,31.669,27.323,24.979,30.000,7.269,2.137,77.506,14.809,10.000
3,13.456,57.052,0.0,26.300,42.785,10.288,189.541,123.524,19.864,70.923,56.874,25.809,24.946,15.551,6.180,5.503,126.785,16.889,73.317
4,16.553,38.521,1.0,21.632,70.622,16.008,99.384,395.248,14.299,48.087,35.802,21.624,38.411,13.696,4.326,11.011,184.122,17.156,27.678


In [19]:
# Summary statistics
behavior.describe()

,PHQ9,Age,Sex,BMI,HRV,Sleep_Quality,Physical_Activity,Caffeine_mg,Stress_Level,Rumination,Loneliness,Social_Support,Neuroticism,Self_Esteem,Alcohol_drinks,Screen_Time,Social_Media,Education_yrs,Income_k
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.00000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.00000,200.000000,200.000000,200.000000
mean,8.224925,37.876045,0.500000,26.945810,53.724775,9.944630,129.81592,219.456310,20.067190,44.507715,42.874075,27.848115,27.444445,18.969445,6.261150,7.19959,110.068985,13.953635,49.260200
std,5.549550,12.892744,0.501255,5.327202,13.342101,4.139252,56.14362,113.219054,7.214863,12.307568,11.280076,8.580074,8.927761,6.048969,4.241435,3.02909,49.968209,2.291382,20.864014
min,0.000000,18.000000,0.000000,15.000000,10.000000,0.000000,0.00000,0.000000,0.866000,22.000000,20.000000,12.000000,0.000000,4.381000,0.000000,0.00000,0.000000,10.000000,10.000000
25%,3.542000,28.260750,0.000000,23.124500,45.468000,7.020250,88.64500,137.006000,14.985250,36.687500,35.693750,22.100750,22.701250,14.876250,2.649750,5.17425,78.585500,12.352500,34.126750
50%,8.008000,38.377000,0.500000,26.806000,54.160500,10.021500,132.23750,208.145000,19.648000,44.861500,42.514000,26.755000,27.720500,19.498000,6.292500,7.27700,108.712500,14.006000,47.737000
75%,12.724000,46.364000,1.000000,30.881500,62.542500,13.178500,168.26525,300.228500,24.958000,52.379750,50.507250,33.636250,32.674250,23.198750,9.628250,9.29575,146.082250,15.325750,64.079500
max,21.931000,70.000000,1.000000,44.051000,97.032000,20.765000,257.85200,504.746000,37.512000,80.963000,71.575000,50.664000,48.000000,30.000000,15.920000,15.27100,258.395000,20.521000,107.075000


Each row is one subject. The table above shows the mean, standard deviation (std), minimum, and maximum for each variable. The 25%/50%/75% rows are **percentiles** -- for example, the 50% row is the median (the middle value).

---

## Part 3: Background and Hypothesis

### Depression and the Salience Network

Research has consistently linked **salience network** dysfunction to major depressive disorder. The salience network -- anchored by the anterior insula and dorsal anterior cingulate cortex (dACC) -- is responsible for detecting and filtering important stimuli. The salience network helps the brain decide what is important to pay attention to and what to ignore. In depression, this filtering process may be disrupted, leading to excessive focus on negative information. Altered connectivity between insular and medial frontal components of this network may disrupt emotional processing and interoceptive awareness.

### Relevant Literature

> Insert: Paper on anterior insula role in interoception and emotional awareness

> Insert: Paper on salience network dysfunction in depression

> <a href="https://www.nature.com/articles/s41586-024-07805-2">Lynch et al 2024:</a>

> <a href="https://pubmed.ncbi.nlm.nih.gov/32680763/">Pimontel et al. 2021:</a> Cortical Thickness of the Salience Network and Change in Apathy Following Antidepressant Treatment for Late-Life Depression, American Journal of Geriatric Psychiatry

Based on this literature, connectivity between insular and medial frontal regions of the salience network may predict individual differences in depression severity.

### Quick Reference: Functional Connectivity Concepts

**Functional connectivity (FC)** measures how correlated the activity is between two brain regions during a resting-state fMRI scan.

**Key terms:**
- **ROI (Region of Interest)**: A specific brain area. This dataset has 216 ROIs from a standard brain atlas.
- **Edge**: A connection between two ROIs. Each edge has a connectivity value per subject. With 216 ROIs, there are 23,220 unique edges.
- **Network**: ROIs are grouped into brain networks (e.g., Default Mode, Salience, Subcortical) based on their function.

**Interpreting Pearson r** (correlation strength):
- |r| < 0.10 -- negligible
- |r| around 0.10-0.20 -- small
- |r| around 0.20-0.30 -- medium
- |r| > 0.30 -- large (rare in neuroimaging)

**r-squared** (r x r) tells you the proportion of variance explained. An r of 0.20 means r-squared = 0.04 -- the brain connection explains about 4% of individual differences.

### Your Turn: Formulate Your Hypothesis

Based on the literature above, write your hypothesis in one sentence. What brain regions do you expect to be connected to PHQ9? In which direction (positive or negative)?

> **H1: Connectivity between insular (Ins) and medial frontal (FrMed) regions of the SalVentAttnA network correlates with PHQ9.**

This is a focused hypothesis: you are testing only edges connecting SalVentAttnA insula regions with SalVentAttnA frontal medial regions -- a small number of specific connections motivated by the literature.

Take a moment to think about *why* you expect this relationship. What does the literature suggest about how salience network disruption might relate to depression?

---

## Part 4: Explore Your Target Regions

Before testing your hypothesis, examine the regions involved and visualize an example edge.

In [ ]:
# What regions are in the SalVentAttnA network?
helpers.list_regions('SalVentAttnA')

In [ ]:
# Overview of all networks
helpers.list_networks()

**Tip:** Use `helpers.describe_regions('SalVentAttnA')` to see decoded names for each region (e.g., what "Ins" and "FrMed" stand for).

In [ ]:
# See decoded region names for the Salience network
helpers.describe_regions('SalVentAttnA')

In [ ]:
# Visualize the overall connectivity structure
helpers.plot_connectome()

In [ ]:
# Zoomed view of SalVentAttnA within-network connectivity
helpers.plot_network_matrix('SalVentAttnA')

### Visualize a Single Edge

Before diving into analysis, let's see what a brain connectivity scatter plot actually looks like. Here we'll plot the connectivity between two subcortical regions against Age -- this is just to get familiar with the visualization, not to test your hypothesis yet.

In [ ]:
# Plot connectivity between two subcortical regions vs Age
# This is just to see what an FC scatter plot looks like
helpers.plot_edge('HIP-lh', 'AMY-lh', 'Age')

---

## Part 5: Your Analysis Toolkit

Before testing your hypothesis, let's learn the analytical tools available to you. Each tool is a legitimate technique that researchers use every day. Understanding these tools will help you make informed decisions when you pre-register your analysis plan in Part 6.

All of these tools work with both `plot_edge()` and the mass testing functions (`test_all_edges()`, `test_network_edges()`).

### 5a. Covariates: Controlling for Confounding Variables

A **covariate** is a variable you account for ("control for") to see whether a relationship still holds after removing its influence. When you control for a variable, the axes show "residualized" values -- what's left of each variable after statistically removing the influence of the covariate.

**Why does this matter?** Sometimes two variables look related, but the apparent relationship is actually driven by a third variable. Let's see an example with the behavioral data first, then apply the same logic to brain data.

In [ ]:
# Is Social_Media use related to Sleep_Quality?
helpers.plot_behavior('Social_Media', 'Sleep_Quality')

# People who use more screens overall may have both more social media use
# AND worse sleep. Let's control for Screen_Time and see what happens:
helpers.plot_behavior('Social_Media', 'Sleep_Quality', covariates=['Screen_Time'])

Notice how the correlation changes after controlling for Screen_Time. The apparent link between social media and sleep quality was largely driven by overall screen time.

The same principle applies to brain data. Let's see how controlling for a behavioral variable changes a brain connectivity relationship:

In [ ]:
# Does hippocampus-visual cortex connectivity predict PHQ9?
helpers.plot_edge('HIP-lh', 'LH_VisCent_ExStr_4', 'PHQ9')

# Now control for Stress_Level and see what happens:
helpers.plot_edge('HIP-lh', 'LH_VisCent_ExStr_4', 'PHQ9',
                  covariates=['Stress_Level'])

Notice how the correlation changed after controlling for Stress_Level. Covariates can **strengthen or weaken** findings. The important thing is to choose your covariates **before** you see your results -- otherwise you might unconsciously pick covariates that make your findings look better.

### 5b. Outlier Handling

An **outlier** is a data point that is unusually far from the rest. **Z-scores** measure how many standard deviations (SD) a value is from the mean. A z-score of 2 means the value is in roughly the most extreme 5% of data; a z-score of 3 is the most extreme 0.3%.

Outliers can have a big impact on correlations. Sometimes a "significant" result is driven entirely by a few extreme values. Other times, outliers can obscure a real effect. Let's see both cases.

You can add an `exclude_outliers` argument to remove extreme values:

In [ ]:
# EXAMPLE 1: A "significant" result that disappears when outliers are removed
print('--- With all data ---')
helpers.plot_edge('LH_DorsAttnB_PostC_2', 'LH_LimbicA_TempPole_2', 'PHQ9')

print('--- After removing outliers (z > 2) ---')
helpers.plot_edge('LH_DorsAttnB_PostC_2', 'LH_LimbicA_TempPole_2', 'PHQ9',
                  exclude_outliers=2)

The correlation went from "significant" to nowhere near significant. Those few extreme data points were creating the illusion of a relationship.

Now let's see the opposite -- outliers *hiding* a real effect:

In [ ]:
# EXAMPLE 2: A real effect that only appears after removing outliers
print('--- With all data ---')
helpers.plot_edge('RH_VisCent_ExStr_5', 'RH_DorsAttnA_SPL_2', 'PHQ9')

print('--- After removing outliers (z > 2) ---')
helpers.plot_edge('RH_VisCent_ExStr_5', 'RH_DorsAttnA_SPL_2', 'PHQ9',
                  exclude_outliers=2)

After removing extreme values, a real relationship emerged. Outliers can work both ways -- they can create false positives or hide true effects. That is why your outlier handling strategy should be decided **in advance** as part of your pre-registration, not adjusted after seeing your results.

### 5c. Subgroup Analysis

Sometimes a relationship looks completely different depending on who you include. Here's a striking example:

In [ ]:
# Overall relationship: neuroticism vs self-esteem
helpers.plot_behavior('Neuroticism', 'Self_Esteem')

# Does the relationship differ by sex?
helpers.plot_behavior('Neuroticism', 'Self_Esteem', subgroup={'Sex': 0})  # women
helpers.plot_behavior('Neuroticism', 'Self_Esteem', subgroup={'Sex': 1})  # men

The overall relationship shows almost no correlation, but when you split by sex, you see that women have a *negative* relationship while men have a *positive* one -- they cancel each other out in the full sample. Subgroup analysis can reveal effects hidden by aggregation. But there is an important trade-off: splitting your sample **reduces your sample size and statistical power**. Only analyze subgroups if you have a strong reason from the literature -- not just because it makes your results look better.

The `subgroup` argument works with brain data too:
```python
helpers.plot_edge('LH_SalVentAttnA_Ins_1', 'RH_SalVentAttnA_FrMed_1', 'PHQ9', subgroup={'Sex': 0})
```

### 5d. Multiple Comparisons

When you test many edges at once, some will appear "significant" just by chance. If you test 100 edges at p < 0.05, you would expect about 5 false positives even if there are no real effects at all.

There are two common ways to correct for this:

- **Bonferroni correction**: Divide your alpha (0.05) by the number of tests. Very conservative -- it controls the chance of *any* false positive. Simple, but can miss real effects when you have many tests.

- **FDR (False Discovery Rate) correction** (Benjamini-Hochberg): Controls the *proportion* of false positives among your significant results. Less conservative, and the standard in neuroimaging research.

Let's see this in action with an example unrelated to your hypothesis. We will test all edges within the Limbic network against PHQ9:

In [ ]:
# Test all within-Limbic edges vs PHQ9 (NOT your hypothesis -- just a demo)
limbic_results = helpers.test_network_edges('Limbic', within=True)
n_limbic = len(limbic_results)

# How many are "significant" without correction?
n_uncorrected = (limbic_results['p'] < 0.05).sum()
print(f'Tested {n_limbic} within-Limbic edges')
print(f'Significant at p < 0.05 (uncorrected): {n_uncorrected}')
print(f'Expected by chance alone: {n_limbic * 0.05:.0f}')

# Now apply FDR correction
limbic_fdr = helpers.test_network_edges('Limbic', within=True, correction='fdr')
n_fdr = (limbic_fdr['p_corrected'] < 0.05).sum()
print(f'\nSignificant after FDR correction: {n_fdr}')
print(f'\nMany "findings" disappear after proper correction!')

This is why multiple comparison correction is essential. Without it, you would report false positives as real findings. In your pre-registration below, we will use **FDR correction** as the default.

### Key Takeaway

You now have a full toolkit: **covariates**, **outlier handling**, and **subgroup analysis**. In the next section, you will formally commit to your analysis choices **before** seeing the results. This is called **pre-registration** -- it is what separates hypothesis-driven science from exploratory analysis.

---

## Part 6: Pre-Register Your Analysis

In real research, scientists often **pre-register** their analysis plan before collecting data. This means writing down exactly what you will test and how -- before you see the results. Pre-registration prevents you from unconsciously adjusting your analysis to get the answer you want.

Fill in each section below. Once you have committed to your plan, you will execute it in Part 8.

### My Pre-Registration

**Hypothesis:** Connectivity between insular (Ins) and medial frontal (FrMed) regions of the SalVentAttnA network correlates with PHQ9.

**Edges to test:** [Which specific edges will you test? e.g., "SalVentAttnA Ins <-> SalVentAttnA FrMed edges"]

**Number of tests:** [How many edges does this include? You will find out when you run the code in Part 8.]

**Correction method:** FDR (Benjamini-Hochberg) -- see explanation below

**Covariates:** [Which covariates will you control for, if any? Justify your choice based on the literature. Write "None" if you will not use covariates.]

**Outlier handling:** [Will you exclude outliers? If so, at what z-score threshold? (e.g., 2 or 3) Write "None" if you will not exclude outliers.]

**Subgroup analysis:** [Will you analyze any subgroups? If so, which? Write "None" if you will analyze the full sample.]

**Significance threshold:** 0.05 (after FDR correction)

---

## Part 7: Practice -- Testing a Known Effect

Before testing your hypothesis, let's practice with a well-established finding.

It is well documented in the neuroscience literature that functional connectivity within the **Default Mode Network** decreases with age. This is one of the most replicated findings in resting-state fMRI research. Let's test this in our data to build confidence with the tools before applying them to your hypothesis.

In [ ]:
# Plot a single Default Mode Network edge vs Age
# These two regions are both in the Default Mode Network:
#   - LH_DefaultA_PFCm_1 = left medial prefrontal cortex
#   - LH_DefaultA_pCunPCC_1 = left precuneus/posterior cingulate
helpers.plot_edge('LH_DefaultA_PFCm_1', 'LH_DefaultA_pCunPCC_1', 'Age')

In [ ]:
# Test all within-Default Mode edges vs Age with FDR correction
dmn_age = helpers.test_network_edges('Default Mode', behavior_col='Age',
                                      correction='fdr', within=True)

n_sig = (dmn_age['p_corrected'] < 0.05).sum()
print(f"\nEdges with significant age-related decline (FDR corrected): {n_sig}")

# Show the top findings
print("\nTop 10 within-Default Mode edges correlated with Age:")
print(dmn_age.head(10)[['ROI_A', 'ROI_B', 'r', 'p', 'p_corrected']].to_string())

You should see mostly **negative correlations** -- as age increases, Default Mode Network connectivity decreases. This is one of the most replicated findings in neuroscience.

If you see significant results after FDR correction, that confirms the tools are working correctly and the data contains real brain-behavior relationships.

Now you are ready to test your own hypothesis.

---

## Part 8: Test Your Hypothesis

Now execute your pre-registered analysis plan. The code below tests your hypothesis edges -- SalVentAttnA insula regions connected to SalVentAttnA medial frontal regions.

In [ ]:
# Step 1: Test edges involving the Salience network
# We focus on Salience because our hypothesis is about SalVentAttnA connectivity
all_results = helpers.test_network_edges('Salience')

In [ ]:
# Step 2: Filter to your specific hypothesis edges
# Keep only rows where one region is SalVentAttnA Ins and the other is SalVentAttnA FrMed
hyp_mask = (
    (all_results['ROI_A'].str.contains('SalVentAttnA') & all_results['ROI_A'].str.contains('Ins') &
     all_results['ROI_B'].str.contains('SalVentAttnA') & all_results['ROI_B'].str.contains('FrMed')) |
    (all_results['ROI_B'].str.contains('SalVentAttnA') & all_results['ROI_B'].str.contains('Ins') &
     all_results['ROI_A'].str.contains('SalVentAttnA') & all_results['ROI_A'].str.contains('FrMed'))
)
hyp_results = all_results[hyp_mask].reset_index(drop=True)

n_hyp_tests = len(hyp_results)
print(f"SalVentAttnA Ins <-> FrMed edges to test: {n_hyp_tests}")
print(f"\nAll results (sorted by p-value):")
hyp_results

### Apply Your Pre-Registered Choices

Modify the code below based on what you wrote in your pre-registration (Part 6). If you chose not to use covariates, outliers, or subgroups, leave them set to `None`.

In [ ]:
# Step 3: Apply your pre-registered analysis choices
# Edit these variables based on what you wrote in your pre-registration:

covariates = None              # e.g., ['Age', 'Sex'] or None for no covariates
outlier_threshold = None       # e.g., 2 or 3, or None for no outlier removal
subgroup = None                # e.g., {'Sex': 0} or None for full sample

# Re-test with your pre-registered choices (only if you specified any)
if covariates is not None or outlier_threshold is not None or subgroup is not None:
    all_results = helpers.test_network_edges('Salience',
                                              covariates=covariates,
                                              exclude_outliers=outlier_threshold,
                                              subgroup=subgroup)

    # Re-filter to hypothesis edges (recompute the mask for the new results)
    hyp_mask = (
        (all_results['ROI_A'].str.contains('SalVentAttnA') & all_results['ROI_A'].str.contains('Ins') &
         all_results['ROI_B'].str.contains('SalVentAttnA') & all_results['ROI_B'].str.contains('FrMed')) |
        (all_results['ROI_B'].str.contains('SalVentAttnA') & all_results['ROI_B'].str.contains('Ins') &
         all_results['ROI_A'].str.contains('SalVentAttnA') & all_results['ROI_A'].str.contains('FrMed'))
    )
    hyp_results = all_results[hyp_mask].reset_index(drop=True)
    n_hyp_tests = len(hyp_results)
    print(f"\nRe-filtered to {n_hyp_tests} hypothesis edges with your pre-registered choices.")
    print(hyp_results[['ROI_A', 'ROI_B', 'r', 'p']].to_string())
else:
    print("No covariates, outlier removal, or subgroup specified.")
    print("Using the results from Step 2 above.")

### Apply Multiple Comparison Correction

Because you are testing multiple edges, you need to correct for multiple comparisons. There are two common approaches:

- **Bonferroni correction**: Divides your alpha (0.05) by the number of tests. Very conservative -- it controls the probability of even one false positive. Simple but can miss real effects.

- **FDR (False Discovery Rate) correction**: Controls the *proportion* of false positives among your significant results. Less conservative than Bonferroni, and widely used in neuroimaging research.

We will use **FDR correction** (Benjamini-Hochberg method), which is the standard approach in neuroimaging and is what the exploratory group used as well.

In [ ]:
from statsmodels.stats.multitest import multipletests

alpha = 0.05

# Apply FDR (Benjamini-Hochberg) correction
reject, p_corrected, _, _ = multipletests(hyp_results['p'], alpha=alpha, method='fdr_bh')
hyp_results['p_corrected'] = p_corrected
hyp_results['significant_fdr'] = reject

# For reference, also compute Bonferroni threshold
bonferroni_threshold = alpha / n_hyp_tests

n_significant = hyp_results['significant_fdr'].sum()

print('=' * 60)
print('MULTIPLE COMPARISON CORRECTION')
print('=' * 60)
print(f'Number of tests: {n_hyp_tests}')
print(f'Method: FDR (Benjamini-Hochberg)')
print(f'For reference -- Bonferroni threshold would be: {bonferroni_threshold:.6f}')
print(f'\nSignificant after FDR correction: {n_significant}')

if n_significant > 0:
    print(f'\nSignificant edges:')
    sig = hyp_results[hyp_results['significant_fdr']]
    for _, row in sig.iterrows():
        print(f"  {row['ROI_A']} <-> {row['ROI_B']}: r={row['r']:.3f}, p={row['p']:.2e}, p_fdr={row['p_corrected']:.2e}")
else:
    print('\nNo edges survived FDR correction.')
    print('Most promising edge:')
    top = hyp_results.iloc[0]
    print(f"  {top['ROI_A']} <-> {top['ROI_B']}: r={top['r']:.3f}, p={top['p']:.2e}")

In [ ]:
# Visualize the top hypothesis-driven findings
def _short_name(roi):
    """Extract readable short name from ROI label."""
    if '-' in roi:  # Subcortical (e.g., NAc-rh)
        return roi
    parts = roi.split('_')
    if len(parts) >= 4:
        return '_'.join(parts[1:-1])  # e.g. SalVentAttnA_Ins
    return roi

if n_significant > 0:
    sig_edges = hyp_results[hyp_results['significant_fdr']]
    n_plots = min(len(sig_edges), 3)
    fig, axes = plt.subplots(1, n_plots, figsize=(5*n_plots, 4))
    if n_plots == 1:
        axes = [axes]

    for idx, (_, row) in enumerate(sig_edges.head(n_plots).iterrows()):
        edge_vals = helpers.get_edge(row['ROI_A'], row['ROI_B'])
        outcome = behavior['PHQ9'].values
        r_val, p_val = pearsonr(edge_vals, outcome)

        axes[idx].scatter(edge_vals, outcome, alpha=0.5, color='steelblue')
        z = np.polyfit(edge_vals, outcome, 1)
        x_line = np.linspace(edge_vals.min(), edge_vals.max(), 100)
        axes[idx].plot(x_line, np.polyval(z, x_line), color='coral', linewidth=2)
        axes[idx].set_xlabel('Functional Connectivity')
        axes[idx].set_ylabel('PHQ9')

        short_a = _short_name(row['ROI_A'])
        short_b = _short_name(row['ROI_B'])
        axes[idx].set_title(f'{short_a} <-> {short_b}\nr = {r_val:.3f}, p = {p_val:.2e}')

    plt.tight_layout()
    plt.show()
else:
    top = hyp_results.iloc[0]
    helpers.plot_edge(top['ROI_A'], top['ROI_B'], 'PHQ9')

### Visualize Results on the Brain

Let's see where your significant edges are located in 3D brain space:

In [ ]:
# Plot significant edges on a glass brain
helpers.plot_glass_brain(hyp_results, p_threshold=0.05)

In [ ]:
# P-value visualization for all hypothesis tests
plt.figure(figsize=(10, 5))

colors = ['mediumseagreen' if sig else 'gray' for sig in hyp_results['significant_fdr']]
plt.scatter(range(len(hyp_results)), hyp_results['p'], c=colors, s=60)
plt.axhline(bonferroni_threshold, color='coral', linestyle='--', linewidth=1,
            label=f'Bonferroni threshold (p={bonferroni_threshold:.2e})')
plt.axhline(0.05, color='orange', linestyle='--', linewidth=1,
            label='Uncorrected alpha = 0.05')
plt.yscale('log')
plt.xlabel('Edge (ranked by p-value)')
plt.ylabel('P-value (log scale)')
plt.title(f'P-values for {n_hyp_tests} SalVentAttnA Insula-FrMed Tests')
plt.legend()
plt.grid(True, alpha=0.2)
plt.tight_layout()
plt.show()

print(f'Green dots = significant after FDR correction')
print(f'Note: FDR correction uses a step-up procedure, not a single threshold line.')
print(f'The Bonferroni line is shown for reference only.')

---

## Part 9: Prepare Your Presentation

For next class, prepare a brief presentation (5-7 minutes) covering:

1. **Your hypothesis** -- What did you predict and why?
2. **Your pre-registered methods** -- Which edges did you test? What correction, covariates, and outlier threshold did you use?
3. **Your results** -- Report ALL tests (not just significant ones). Show visualizations.
4. **Why you believe your findings** -- What makes you confident? Do the brain regions make sense?
5. **How would you convince a skeptic?** -- What evidence would you point to?

**Important**: Be honest about null results! Rigorous science means reporting what you found, not just what you hoped to find.

In [ ]:
# Save your key figures for your presentation
# Re-plot your strongest finding and save it
if n_significant > 0:
    top = hyp_results[hyp_results['significant_fdr']].iloc[0]
    helpers.plot_edge(top['ROI_A'], top['ROI_B'], 'PHQ9',
                      covariates=covariates,
                      exclude_outliers=outlier_threshold,
                      subgroup=subgroup)
    plt.savefig('my_finding.png', dpi=150, bbox_inches='tight')
    print("Figure saved as 'my_finding.png'")
    print("Download this file for your presentation.")
else:
    top = hyp_results.iloc[0]
    helpers.plot_edge(top['ROI_A'], top['ROI_B'], 'PHQ9',
                      covariates=covariates,
                      exclude_outliers=outlier_threshold,
                      subgroup=subgroup)
    plt.savefig('my_finding.png', dpi=150, bbox_inches='tight')
    print("Figure saved as 'my_finding.png'")
    print("Even null results are worth presenting!")

---
---

# DAY 2 STARTS HERE

---

---

## Part 10: Reconnect and Reflect

If your Colab runtime disconnected since Day 1, run the cell below to reload everything. If your runtime is still active, you can skip it.

In [ ]:
# Day 2 Setup: Re-run if your runtime disconnected
import subprocess, sys
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'nilearn', '-q'])

import os, urllib.request
base_url = 'https://raw.githubusercontent.com/cmahlen/python-stats-demo/main/'
files_needed = [
    'lab_helpers.py', 'atlas_labels.txt', 'data/roi_mni_coords.npy',
    'data/depression_discovery.npz', 'data/depression_validation.npz',
]
os.makedirs('data', exist_ok=True)
for f in files_needed:
    if not os.path.exists(f):
        urllib.request.urlretrieve(base_url + f, f)

import lab_helpers as helpers
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from statsmodels.stats.multitest import multipletests

helpers.load_dataset('depression', 'discovery')
behavior = helpers.get_behavior()

# Re-run your hypothesis test
all_results = helpers.test_network_edges('Salience')
hyp_mask = (
    (all_results['ROI_A'].str.contains('SalVentAttnA') & all_results['ROI_A'].str.contains('Ins') &
     all_results['ROI_B'].str.contains('SalVentAttnA') & all_results['ROI_B'].str.contains('FrMed')) |
    (all_results['ROI_B'].str.contains('SalVentAttnA') & all_results['ROI_B'].str.contains('Ins') &
     all_results['ROI_A'].str.contains('SalVentAttnA') & all_results['ROI_A'].str.contains('FrMed'))
)
hyp_results = all_results[hyp_mask].reset_index(drop=True)
n_hyp_tests = len(hyp_results)

# Re-apply your pre-registered choices
covariates = None              # <-- Match what you used on Day 1
outlier_threshold = None       # <-- Match what you used on Day 1
subgroup = None                # <-- Match what you used on Day 1

if covariates is not None or outlier_threshold is not None or subgroup is not None:
    all_results = helpers.test_network_edges('Salience',
                                              covariates=covariates,
                                              exclude_outliers=outlier_threshold,
                                              subgroup=subgroup)
    hyp_mask = (
        (all_results['ROI_A'].str.contains('SalVentAttnA') & all_results['ROI_A'].str.contains('Ins') &
         all_results['ROI_B'].str.contains('SalVentAttnA') & all_results['ROI_B'].str.contains('FrMed')) |
        (all_results['ROI_B'].str.contains('SalVentAttnA') & all_results['ROI_B'].str.contains('Ins') &
         all_results['ROI_A'].str.contains('SalVentAttnA') & all_results['ROI_A'].str.contains('FrMed'))
    )
    hyp_results = all_results[hyp_mask].reset_index(drop=True)
    n_hyp_tests = len(hyp_results)

alpha = 0.05
reject, p_corrected, _, _ = multipletests(hyp_results['p'], alpha=alpha, method='fdr_bh')
hyp_results['p_corrected'] = p_corrected
hyp_results['significant_fdr'] = reject
bonferroni_threshold = alpha / n_hyp_tests
n_significant = hyp_results['significant_fdr'].sum()

def _short_name(roi):
    """Extract readable short name from ROI label."""
    if '-' in roi:
        return roi
    parts = roi.split('_')
    if len(parts) >= 4:
        return '_'.join(parts[1:-1])
    return roi

print(f"\nReloaded! {n_significant} significant edges (FDR corrected)")

### Reflection Questions

Before we test your findings in the validation data, think about these questions:

1. **How confident are you that your findings will replicate?**
   - What would make you more or less confident?

2. **What could go wrong?**
   - Even with Bonferroni correction, could your results still be false positives?

3. **How is your approach different from the exploratory group's?**
   - How many tests did you run compared to them?
   - Did you choose your analysis plan before or after seeing the results?

4. **Effect sizes**
   - How strong were the correlations you found?
   - Are these practically meaningful for understanding depression?

---

## Part 11: Validation

### Test your findings in the independent validation set

True effects should replicate in new data. You will now re-run your **exact pre-registered analysis** on the validation dataset.

In [ ]:
# Load the validation dataset
helpers.load_dataset('depression', 'validation')

The cell below tests each of your hypothesis edges in the validation dataset and compares the results to your discovery findings. An edge "replicates" if it is significant in the same direction in both datasets. You do not need to understand every line of code -- just run the cell and read the output.

In [ ]:
import pandas as pd

# Test ALL hypothesis edges in validation using the SAME analysis choices
val_behavior = helpers.get_behavior()
val_outcome = val_behavior['PHQ9'].values

validation_results = []
for _, row in hyp_results.iterrows():
    edge_vals = helpers.get_edge(row['ROI_A'], row['ROI_B'])
    r_val, p_val = pearsonr(edge_vals, val_outcome)

    sig_disc = row['significant_fdr']
    # Check both significance AND direction (sign of r must match)
    same_direction = (r_val * row['r']) > 0
    if sig_disc:
        if p_val < 0.05 and same_direction:
            replicated = 'YES'
        elif p_val < 0.05 and not same_direction:
            replicated = 'FLIPPED'
        else:
            replicated = 'NO'
    else:
        replicated = 'N/A'

    validation_results.append({
        'ROI_A': row['ROI_A'],
        'ROI_B': row['ROI_B'],
        'Disc_r': row['r'],
        'Disc_p': row['p'],
        'Sig_Disc': sig_disc,
        'Val_r': r_val,
        'Val_p': p_val,
        'Replicated': replicated,
    })

val_df = pd.DataFrame(validation_results)

print('=' * 80)
print('VALIDATION RESULTS: SalVentAttnA Insula-FrMed Edges')
print('=' * 80)
print(f'Validation uses uncorrected p < 0.05 + same direction as replication criterion\n')

display_df = val_df[['ROI_A', 'ROI_B', 'Disc_r', 'Disc_p', 'Sig_Disc', 'Val_r', 'Val_p', 'Replicated']]
display_df.index = range(1, len(display_df) + 1)
print(display_df.to_string())

if n_significant > 0:
    n_rep = (val_df['Replicated'] == 'YES').sum()
    print(f"\n{'=' * 80}")
    print(f'REPLICATION: {n_rep}/{n_significant} significant findings replicated')
    print(f"{'=' * 80}")

n_flipped = (val_df['Replicated'] == 'FLIPPED').sum()
if n_flipped > 0:
    print(f'WARNING: {n_flipped} finding(s) were significant but in the OPPOSITE direction!')
    print('A flipped direction means the effect is not replicating -- it is noise.')

The cell below creates side-by-side scatter plots comparing your discovery and validation results. The left plot shows the discovery set; the right plot shows the validation set. If the finding replicated, both plots should show a similar pattern.

In [ ]:
# Side-by-side visualization for significant finding(s)
if n_significant > 0:
    sig_rows = val_df[val_df['Sig_Disc'] == True]

    for _, row in sig_rows.iterrows():
        fig, axes = plt.subplots(1, 2, figsize=(13, 5))

        # Discovery
        helpers.load_dataset('depression', 'discovery')
        disc_edge = helpers.get_edge(row['ROI_A'], row['ROI_B'])
        disc_out = helpers.get_behavior()['PHQ9'].values
        r_d, p_d = pearsonr(disc_edge, disc_out)

        axes[0].scatter(disc_edge, disc_out, alpha=0.5, color='steelblue')
        z = np.polyfit(disc_edge, disc_out, 1)
        x_line = np.linspace(disc_edge.min(), disc_edge.max(), 100)
        axes[0].plot(x_line, np.polyval(z, x_line), color='coral', linewidth=2)
        axes[0].set_xlabel('Functional Connectivity')
        axes[0].set_ylabel('PHQ9')
        axes[0].set_title(f'Discovery Set\nr = {r_d:.3f}, p = {p_d:.2e}')

        # Validation
        helpers.load_dataset('depression', 'validation')
        val_edge = helpers.get_edge(row['ROI_A'], row['ROI_B'])
        val_out = helpers.get_behavior()['PHQ9'].values
        r_v, p_v = pearsonr(val_edge, val_out)

        axes[1].scatter(val_edge, val_out, alpha=0.5, color='mediumseagreen')
        z = np.polyfit(val_edge, val_out, 1)
        x_line = np.linspace(val_edge.min(), val_edge.max(), 100)
        axes[1].plot(x_line, np.polyval(z, x_line), color='coral', linewidth=2)
        axes[1].set_xlabel('Functional Connectivity')
        axes[1].set_ylabel('PHQ9')
        axes[1].set_title(f'Validation Set\nr = {r_v:.3f}, p = {p_v:.2e}')

        short_a = _short_name(row['ROI_A'])
        short_b = _short_name(row['ROI_B'])
        fig.suptitle(f'{short_a} <-> {short_b}: Discovery vs Validation', fontsize=13)
        plt.tight_layout()
        plt.show()

        if row['Replicated'] == 'YES':
            print(f'REPLICATED in validation set!')
        else:
            print(f'Did not replicate in validation set.')
        print()
else:
    print('No significant findings to visualize.')

### Your Turn: Test a Non-Significant Edge in Validation

Pick one edge from your hypothesis set that was NOT significant in discovery. Does it look any different in validation? This helps build intuition about what "noise" looks like.

<details>
<summary>Hint: Example code</summary>

```python
# Pick the last (least significant) edge from your hypothesis results
nonsig = hyp_results.iloc[-1]
helpers.plot_edge(nonsig['ROI_A'], nonsig['ROI_B'], 'PHQ9')
```
</details>

In [ ]:
# Your Turn: test a non-significant edge here
